<a href="https://colab.research.google.com/github/dasalgadob/handson-machine-learning/blob/master/3Imageretrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import stack
#from keras.datasets import mnist
from keras.datasets import mnist
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import os

Using TensorFlow backend.


In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

In [4]:
!pip install tensorboardcolab

### Chapter 1 Previous code to chapter 3

In [5]:
## Loading
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 400

In [0]:
## Placeholders for input data and targets
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

In [0]:
weights = tf.Variable(tf.random_normal([input_size, no_classes]))
bias = tf.Variable(tf.random_normal([no_classes]))

In [0]:
logits = tf.matmul(x_input, weights) + bias

In [0]:
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
labels=y_input, logits=logits)
loss_operation = tf.reduce_mean(softmax_cross_entropy)
optimiser = tf.train.GradientDescentOptimizer(
learning_rate=0.5).minimize(loss_operation)

In [0]:
### Training the model with data
session = tf.Session()
session.run(tf.global_variables_initializer())

for batch_no in range(total_batches):
  mnist_batch = mnist_data.train.next_batch(batch_size)
  _, loss_value = session.run([optimiser, loss_operation], feed_dict={
  x_input: mnist_batch[0],
  y_input: mnist_batch[1]
  })
  #print(loss_value)

In [12]:
predictions = tf.argmax(logits, 1)
correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions,
tf.float32))
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
accuracy_value = session.run(accuracy_operation, feed_dict={
x_input: test_images,
y_input: test_labels
})
print('Accuracy : ', accuracy_value)
session.close()

Accuracy :  0.8392


In [0]:
### Multilayer convolutional network

def add_variable_summary(tf_variable, summary_name):
  with tf.name_scope(summary_name + '_summary'):
    mean = tf.reduce_mean(tf_variable)
    tf.summary.scalar('Mean', mean)
    with tf.name_scope('standard_deviation'):
      standard_deviation = tf.sqrt(tf.reduce_mean( tf.square(tf_variable - mean)))
    tf.summary.scalar('StandardDeviation', standard_deviation)
    tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
    tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
    tf.summary.histogram('Histogram', tf_variable)

In [0]:
x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1],
name='input_reshape') ## -1 refers that the batch size could be any

In [0]:
def convolution_layer(input_layer, filters, kernel_size=[3, 3], activation=tf.nn.relu):
  layer = tf.layers.conv2d(
  inputs=input_layer,
  filters=filters,
  kernel_size=kernel_size,
  activation=activation,
  )
  add_variable_summary(layer, 'convolution')
  return layer

In [0]:
def pooling_layer(input_layer, pool_size=[2, 2], strides=2):
  layer = tf.layers.max_pooling2d(
  inputs=input_layer,
  pool_size=pool_size,
  strides=strides
  )
  add_variable_summary(layer, 'pooling')
  return layer

In [0]:
def dense_layer(input_layer, units, activation=tf.nn.relu):
  layer = tf.layers.dense(
  inputs=input_layer,
  units=units,
  activation=activation
  )
  add_variable_summary(layer, 'dense')
  return layer

In [0]:
convolution_layer_1 = convolution_layer(x_input_reshape, 64)
pooling_layer_1 = pooling_layer(convolution_layer_1)
convolution_layer_2 = convolution_layer(pooling_layer_1, 128)
pooling_layer_2 = pooling_layer(convolution_layer_2)
flattened_pool = tf.reshape(pooling_layer_2, [-1, 5 * 5 * 128],
name='flattened_pool')
dense_layer_bottleneck = dense_layer(flattened_pool, 1024)

In [0]:
dropout_bool = tf.placeholder(tf.bool)
dropout_layer = tf.layers.dropout(
inputs=dense_layer_bottleneck,
rate=0.4,
training=dropout_bool
)

In [0]:
logits = dense_layer(dropout_layer, no_classes)

In [0]:
with tf.name_scope('loss'):
  softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
  labels=y_input, logits=logits)
  loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
  tf.summary.scalar('loss', loss_operation)

In [0]:
with tf.name_scope('optimiser'):
  optimiser = tf.train.AdamOptimizer().minimize(loss_operation)

In [23]:
with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    predictions = tf.argmax(logits, 1)
    correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
  with tf.name_scope('accuracy'):
    accuracy_operation = tf.reduce_mean( tf.cast(correct_predictions, tf.float32))
tf.summary.scalar('accuracy', accuracy_operation)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

### Tensorboard

In [24]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-01-28 14:47:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 34.199.255.1, 35.173.3.255, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  3.46MB/s    in 1.5s    

2019-01-28 14:47:06 (3.46 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [42]:
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://4e333fd3.ngrok.io


In [0]:
#LOG_DIR = './log'
#get_ipython().system_raw(
#    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#    .format(LOG_DIR)
#)

In [0]:
#get_ipython().system_raw('./ngrok http 6006 &')

In [28]:
#! curl -s http://localhost:4040/api/tunnels | python3 -c \
#    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://602d2557.ngrok.io


In [0]:
#!tensorboard --logdir=/tmp/train --port 6007


In [0]:
merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('./Graph/train', session.graph)
test_summary_writer = tf.summary.FileWriter('./Graph/test')

In [0]:
session = tf.Session()
session.run(tf.global_variables_initializer())

test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
for batch_no in range(400):
  mnist_batch = mnist_data.train.next_batch(32)
  train_images, train_labels = mnist_batch[0], mnist_batch[1]
  _, merged_summary = session.run([optimiser, merged_summary_operation],
  feed_dict={
  x_input: train_images,
  y_input: train_labels,
  dropout_bool: True
  })
  train_summary_writer.add_summary(merged_summary, batch_no)
  if batch_no % 10 == 0:
    merged_summary, _ = session.run([merged_summary_operation,
    accuracy_operation], feed_dict={
    x_input: test_images,
    y_input: test_labels,
    dropout_bool: False
    })
    test_summary_writer.add_summary(merged_summary, batch_no)

## Embedding visualization

In [31]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 14s 1us/step


In [32]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
print(mnist_data[::1])

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5321fc4b00>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5321fc4ba8>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5321fc4a58>)


In [0]:
no_embedding_data = 1000
embedding_variable = tf.Variable(stack( mnist_data.test.images[:no_embedding_data], axis=0), trainable= False)

### MNIST test data to metadata file for visualization

In [0]:
metadata_path = '/metadata.tsv'
work_dir = ''

with open(metadata_path, 'w') as metadata_file:
  for i in range(no_embedding_data):
    metadata_file.write('{}\n'.format( np.nonzero(mnist_data.test.labels[::1])[1:][0][i]))
    
    
projector_config =projector.ProjectorConfig()
embedding_projection = projector_config.embeddings.add()
embedding_projection.tensor_name = embedding_variable.name
embedding_projection.metadata_path = metadata_path
embedding_projection.sprite.image_path = os.path.join( work_dir + 'mnist_10k_sprite.png')
embedding_projection.sprite.single_image_dim.extend([28,28])

In [0]:
#projector.visualize_embeddings(train_summary_writer, projector_config)
#tf.train.Saver().save(session, '/tmp/train/model.ckpt', global_step=1)